In [3]:
import math

In [4]:
class Query():
    
    param_name = ""
    param_skip = 0
    param_take = math.inf
    param_includes = []
    param_filters = {}
    param_orderby = ""
    param_orderbydescending = False
    index = 0
    data = []
    
    # for testing only --
    taken = 0
    skipped = 0
    current = 0
    # --
    
    def __init__(self):
        self.param_name = ""
        self.param_skip = 0
        self.param_take = 0
        self.param_includes = []
        self.param_filters = {}
    
    def skip(self, n):
        if self.param_skip > 0: raise ValueError('Skip already set.')
        self.param_skip = n
        self.index = n
        return self
    
    def take(self, n):
        if self.param_take > 0: raise ValueError('Take already set.')
        self.param_take = n
        return self
    
    def include(self, name):
        self.param_includes.append(name)
        return self
    
    def filter(self, property, value):
        self.param_filters[property] = value
        return self
    
    def orderBy(self, property, descending = False):
        self.param_orderby = property
        self.param_orderbydescending = descending
        return self
        
    def orderByDescending(self, property):
        return self.orderBy(property, True)
    
    def __iter__(self):
        return self
    
    def load(self):
        print('Lazy load!')
        self.data = range(self.param_skip + self.param_take)
        
    def url_qry_params(self):
        params = {}
        if self.param_skip > 0: params['skip'] = self.param_skip
        if self.param_take > 0: params['limit'] = self.param_take
        if self.param_orderby: params['orderby'] = self.param_orderby
        if self.param_orderbydescending: params['descending'] = 'true'
        if len(self.param_includes) > 0: params['include'] = ",".join(self.param_includes)
        return ('{}?'.format(self.param_name) 
                    + '&'.join(['{}={}'.format(k,v) for k,v in dict(params, **self.param_filters).items()] )
               ).lower()
    
    def __next__(self): 
        if self.data == []: self.load()
        if self.taken >= self.param_take: raise StopIteration
        try: result = self.data[self.index]
        except IndexError: raise StopIteration
        self.index += 1
        self.taken += 1
        return result
    
def get(name):
    q = Query()
    q.param_name = name
    return q

In [2]:
# Required packages:
# pip3 install requests pandas
import requests
import json
from enum import Enum
import pandas as pd

def head(s): return s[0] if len(s) > 0 else []
def tail(s): return s[1:]
def reverse(s): return s[::-1]
def last(s): return s[-1:]
def init(s): return s[0:len(s)-1]
def take(s, n): return s[:n]
def drop(s, n): return s[n:]
def product(s): return reduce(lambda x, y: x * y, s)

class TextType(Enum):
    Description = 0
    ExtendedDescription = 1
    MoreInfo = 2

class ElfskotApi():

    base_address = 'https://api.elfskot.cloud/api/2/'

    def __init__(self, application_id, secret):
        self.application_id = application_id
        self.secret = secret
        self.get_token()

    def get_token(self):
        payload = { 'clientId': self.application_id, 'secret': self.secret}
        result = requests.post(self.get_url('auth/elfskotconnectlogin'), json=payload)
        self.check_result(result)
        self.token = json.loads(result.text)['accessToken']

    def check_result(self, result):
        if result.status_code != 200:
            raise ValueError('Error: API returned status code {}'.format(result.status_code))
        return True

    def get_auth_header(self): return {'Authorization': 'bearer {}'.format(self.token)}
    def get_url(self, endpoint): return self.base_address + endpoint

    def http_return_if_valid(self, result):
        self.check_result(result)
        return json.loads(result.text)

    def http_get(self, endpoint):
        return self.http_return_if_valid(
            requests.get(self.get_url(endpoint), headers=self.get_auth_header())
        )

    def http_post(self, endpoint, o):
        return self.http_return_if_valid(
            requests.post(self.get_url(endpoint), json=o, headers=self.get_auth_header())
        )

    def http_put(self, endpoint, o):
        return self.http_return_if_valid(
            requests.put(self.get_url(endpoint), json=o, headers=self.get_auth_header())
        )

    def http_delete(self, endpoint, key):
        result = requests.delete(self.get_url(endpoint) + '/{}'.format(key), headers=self.get_auth_header())
        if result.status_code != 200: 
            raise ValueError('Error: API returned status code {}'.format(result.status_code))

    def all(self, endpoint): return self.http_get(endpoint)
    def get(self, endpoint, k): return head(self.http_get('{}?id={}'.format(endpoint, k)))
    def find(self, endpoint, p, v): return self.http_get('{}?{}={}'.format(endpoint, p, v))
    def first(self, endpoint, p, v): return head(self.find(endpoint, p, v))
    def new(self, endpoint, o): return self.http_post(endpoint, o)
    def update(self, endpoint, o): return self.http_put(endpoint, o)
    def delete(self, endpoint, k): self.http_delete(endpoint, k)  
    def help(self, endpoint): print('Model for {}:\r\n{}'
                                    .format(endpoint, list(self.first(endpoint, 'id', '').keys())))